In [69]:
# import libraries
import pandas as pd
import os
os.getcwd()

'/Users/benberinsky/Desktop/data_science/data_science_y2s2/DS-3021/labs'

In [70]:
#read in the data, get an idea of what they are showing
nfl_data = pd.read_csv("../data/nfl2008_fga.csv")
nfl_data.head()

,GameDate,AwayTeam,HomeTeam,qtr,min,sec,kickteam,def,down,togo,...,distance,homekick,kickdiff,timerem,offscore,defscore,season,GOOD,Missed,Blocked
0,20081130,IND,CLE,1,47,2,IND,CLE,4.0,11.0,...,30,0,-3,2822,0,3,2008,1,0,0
1,20081005,IND,HOU,1,54,47,IND,HOU,4.0,3.0,...,46,0,0,3287,0,0,2008,1,0,0
2,20081228,TEN,IND,1,45,20,IND,TEN,4.0,3.0,...,28,1,7,2720,7,0,2008,1,0,0
3,20081012,BAL,IND,1,45,42,IND,BAL,4.0,1.0,...,37,1,14,2742,14,0,2008,1,0,0
4,20080907,CHI,IND,1,50,56,IND,CHI,4.0,21.0,...,39,1,0,3056,0,0,2008,1,0,0


#### Question: Does the quarter a field goal is kicked in have an effect on make percentage?
An easy way to handle this is to identify function calls starting at the end of the question and working backwards. 

#### Pseudocode:
  - Group field goals by quarter(after making them categorical)
  - Calculate total make percentage for games
  - Calculate make percentage for each quarter
  - Display make percentage for both total game and individual quarters
  - Arrange in descending order by quarter

In [71]:
#print out the info about the data to understand structure of variables
print(nfl_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GameDate  1039 non-null   int64  
 1   AwayTeam  1039 non-null   object 
 2   HomeTeam  1039 non-null   object 
 3   qtr       1039 non-null   int64  
 4   min       1039 non-null   int64  
 5   sec       1039 non-null   int64  
 6   kickteam  1039 non-null   object 
 7   def       1039 non-null   object 
 8   down      1037 non-null   float64
 9   togo      1037 non-null   float64
 10  kicker    1039 non-null   int64  
 11  ydline    1039 non-null   int64  
 12  name      1039 non-null   object 
 13  distance  1039 non-null   int64  
 14  homekick  1039 non-null   int64  
 15  kickdiff  1039 non-null   int64  
 16  timerem   1039 non-null   int64  
 17  offscore  1039 non-null   int64  
 18  defscore  1039 non-null   int64  
 19  season    1039 non-null   int64  
 20  GOOD      1039 non-null   int6

In [72]:
#Make quarter a categorical variable instead of int, 1-5
nfl_data['qtr'] = nfl_data['qtr'].astype("category")
print(nfl_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   GameDate  1039 non-null   int64   
 1   AwayTeam  1039 non-null   object  
 2   HomeTeam  1039 non-null   object  
 3   qtr       1039 non-null   category
 4   min       1039 non-null   int64   
 5   sec       1039 non-null   int64   
 6   kickteam  1039 non-null   object  
 7   def       1039 non-null   object  
 8   down      1037 non-null   float64 
 9   togo      1037 non-null   float64 
 10  kicker    1039 non-null   int64   
 11  ydline    1039 non-null   int64   
 12  name      1039 non-null   object  
 13  distance  1039 non-null   int64   
 14  homekick  1039 non-null   int64   
 15  kickdiff  1039 non-null   int64   
 16  timerem   1039 non-null   int64   
 17  offscore  1039 non-null   int64   
 18  defscore  1039 non-null   int64   
 19  season    1039 non-null   int64   
 20  GOOD    

In [73]:
#create new dataframe where kicks are grouped by quarter, shows makes/misses/blocks
qtr_groupings = nfl_data.groupby('qtr')
qtr_groupings = qtr_groupings.agg({'GOOD': 'sum', 'Missed': 'sum', 'Blocked': 'sum'})
qtr_groupings

/var/folders/n6/pgk9bwwd15g8ch5_lb0_0gvm0000gn/T/ipykernel_61307/486530792.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  qtr_groupings = nfl_data.groupby('qtr')


,GOOD,Missed,Blocked
qtr,,,
1,192,22,3
2,316,59,9
3,154,28,4
4,226,29,6
5,12,1,0


In [78]:
#Calculates overall total make %
make_pct=qtr_groupings.GOOD.sum()/(qtr_groupings.GOOD.sum() + qtr_groupings.Missed.sum() + qtr_groupings.Blocked.sum())
print("Overall make pct:", make_pct.round(4))

Overall make pct: 0.8483


In [75]:
#Calculates the make percentage for each quarter, adds total in a new column
x3 = qtr_groupings.assign(grouped_make_pct = lambda x: (qtr_groupings['GOOD'] / (qtr_groupings['GOOD'] + qtr_groupings['Missed'] + qtr_groupings['Blocked'])))
x3 = x3.assign(total_make_pct = make_pct)
x3.head()

,GOOD,Missed,Blocked,grouped_make_pct,total_make_pct
qtr,,,,,
1,192,22,3,0.884793,0.848256
2,316,59,9,0.822917,0.848256
3,154,28,4,0.827957,0.848256
4,226,29,6,0.865900,0.848256
5,12,1,0,0.923077,0.848256


In [76]:
#Sorts by quarter pct
x3_sorted = x3.sort_values(by='grouped_make_pct', ascending=False)
x3_sorted

,GOOD,Missed,Blocked,grouped_make_pct,total_make_pct
qtr,,,,,
5,12,1,0,0.923077,0.848256
1,192,22,3,0.884793,0.848256
4,226,29,6,0.865900,0.848256
3,154,28,4,0.827957,0.848256
2,316,59,9,0.822917,0.848256


#### Function construction

In [81]:
def fg_pct_qrts(x):
    x['qtr'] = x['qtr'].astype("category")
    x1 = x.groupby('qtr')
    x1 = x1.agg({'GOOD': 'sum', 'Missed': 'sum', 'Blocked': 'sum'})
    make_pct_1 = x1.GOOD.sum()/(x1.GOOD.sum() + x1.Missed.sum() + x1.Blocked.sum())
    x2 = x1.assign(grouped_make_pct = lambda x: (x1['GOOD'] / (x1['GOOD'] + x1['Missed'] + x1['Blocked'])))
    x2 = x2.assign(total_make_pct = make_pct)
    x2_sorted = x2.sort_values(by='grouped_make_pct', ascending=False)
    return(x2)

fg_pct_qrts(nfl_data)


/var/folders/n6/pgk9bwwd15g8ch5_lb0_0gvm0000gn/T/ipykernel_61307/476320637.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  x1 = x.groupby('qtr')


,GOOD,Missed,Blocked,grouped_make_pct,total_make_pct
qtr,,,,,
1,192,22,3,0.884793,0.848256
2,316,59,9,0.822917,0.848256
3,154,28,4,0.827957,0.848256
4,226,29,6,0.865900,0.848256
5,12,1,0,0.923077,0.848256
